In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select

import mysql.connector
import pandas as pd
from datetime import datetime

In [2]:
# CVS 파일 읽기
df = pd.read_excel("../data/예금은행 대출금리_20248130131449.xlsx", 
                   header=None,
                   usecols="A:MG",
                   skiprows=25,
                   nrows = 5,
                    dtype=str
                   )

# 데이터 전처리
df.reset_index(drop=True, inplace=True)
df_loan_interest_rate= df.transpose()
df_loan_interest_rate.rename(columns=df.iloc[0], inplace=True)
df_loan_interest_rate.drop([0], inplace=True)
df_loan_interest_rate.reset_index(drop=True, inplace=True)
df_loan_interest_rate.columns = ['date', 'average_loan', 'corporate_loan', 'household_loan', 'public_and_other_sector_loan']

df_loan_interest_rate.loc[[9, 21, 33, 45], 'date'] = ['1996.10', '1997.10', '1998.10', '1999.10']

# date 형식 변환
df_loan_interest_rate['date'] = pd.to_datetime(df_loan_interest_rate['date'], format="%Y.%m")

# 데이터프레임을 엑셀 파일로 저장
df_loan_interest_rate.to_excel("../data/loan_interest_rate.xlsx", index=False, engine='openpyxl')

print(df_loan_interest_rate)


          date average_loan corporate_loan household_loan  \
0   1996-01-01        11.43          11.14          12.52   
1   1996-02-01        11.47          11.21           12.5   
2   1996-03-01        11.37          11.11          12.46   
3   1996-04-01        11.11          10.91          12.13   
4   1996-05-01        11.24           11.1          12.09   
..         ...          ...            ...            ...   
339 2024-04-01         4.77           4.88           4.48   
340 2024-05-01         4.78            4.9           4.49   
341 2024-06-01         4.71           4.88           4.26   
342 2024-07-01         4.55           4.78           4.06   
343 2024-08-01         4.48           4.67           4.08   

    public_and_other_sector_loan  
0                           9.77  
1                          10.06  
2                          10.36  
3                          11.27  
4                          11.31  
..                           ...  
339                   

/home/naon/dev_ws/eda_repo/.venv/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
import mysql.connector

conn = mysql.connector.connect(
    host = "eda-database.c3oek28e8yta.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "whskdhs",
    password = "whskdhs1234",
    database = "workspace"
)

cursor = conn.cursor(buffered=True)

df = pd.read_excel('../data/loan_interest_rate.xlsx')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          344 non-null    datetime64[ns]
 1   average_loan                  344 non-null    float64       
 2   corporate_loan                344 non-null    float64       
 3   household_loan                344 non-null    float64       
 4   public_and_other_sector_loan  344 non-null    float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 13.6 KB


In [5]:
# 데이터베이스에 저장
def insert_data_to_loan_interest_rate(conn, cursor, df):
    try:
        for i, row, in df.iterrows():
            sql_loan = "INSERT INTO loan_interest_rate VALUES (%s, %s, %s, %s, %s)"
            values_loan = (row['date'], row['average_loan'], row['corporate_loan'], row['household_loan'], row['public_and_other_sector_loan'])
            cursor.execute(sql_loan, values_loan)
            print(f"Inserted row {i+1} into article: {row['date']}")
            conn.commit()
    
    except mysql.connector.Error as error:
        print(f"Error inserting data: {error}")

In [6]:
insert_data_to_loan_interest_rate(conn, cursor, df)

Inserted row 1 into article: 1996-01-01 00:00:00
Inserted row 2 into article: 1996-02-01 00:00:00
Inserted row 3 into article: 1996-03-01 00:00:00
Inserted row 4 into article: 1996-04-01 00:00:00
Inserted row 5 into article: 1996-05-01 00:00:00
Inserted row 6 into article: 1996-06-01 00:00:00
Inserted row 7 into article: 1996-07-01 00:00:00
Inserted row 8 into article: 1996-08-01 00:00:00
Inserted row 9 into article: 1996-09-01 00:00:00
Inserted row 10 into article: 1996-10-01 00:00:00
Inserted row 11 into article: 1996-11-01 00:00:00
Inserted row 12 into article: 1996-12-01 00:00:00
Inserted row 13 into article: 1997-01-01 00:00:00
Inserted row 14 into article: 1997-02-01 00:00:00
Inserted row 15 into article: 1997-03-01 00:00:00
Inserted row 16 into article: 1997-04-01 00:00:00
Inserted row 17 into article: 1997-05-01 00:00:00
Inserted row 18 into article: 1997-06-01 00:00:00
Inserted row 19 into article: 1997-07-01 00:00:00
Inserted row 20 into article: 1997-08-01 00:00:00
Inserted 